# 04 Naive transfer and forgetting
Hypothesis: aggressive fine-tuning increases drift and forgetting risk.

### Expected Outcome
Drift and gradient norm should rise while source retention declines.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.notebook_transfer import TransferNotebookLab

LAB = TransferNotebookLab.from_root(ROOT)
FAST_DEV_RUN = False

In [ ]:
LAB.run(
    base_name='transfer_safe_vs_naive.yaml',
    notebook_tag='04_naive',
    fast_dev_run=FAST_DEV_RUN,
    overrides={
        'methods': ['naive_finetune'],
        'train': {'lr_target': 0.04, 'target_epochs': 16},
    },
)
naive = LAB.read_method('naive_finetune')
naive[['epoch', 'target_test_acc', 'source_retention_acc', 'feature_drift', 'grad_norm']].tail()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12.5, 3.4))
axes[0].plot(naive['epoch'], naive['target_test_acc'], marker='o')
axes[0].set_title('target accuracy')
axes[1].plot(naive['epoch'], naive['source_retention_acc'], marker='o')
axes[1].set_title('source retention')
axes[2].plot(naive['epoch'], naive['feature_drift'], marker='o')
axes[2].set_title('feature drift')
for ax in axes:
    ax.set_xlabel('epoch')
    ax.grid(alpha=0.25)
LAB.savefig(fig, '04_naive_failure_dashboard.png')

fig, ax = plt.subplots(figsize=(6.2, 3.6))
ax.plot(naive['epoch'], naive['grad_norm'], marker='o', color='#E45756')
ax.set_title('Naive fine-tuning gradient norm')
ax.set_xlabel('epoch')
ax.set_ylabel('grad_norm')
ax.grid(alpha=0.25)
LAB.savefig(fig, '04_naive_grad_norm.png')

### Reading This Pattern
Rising `feature_drift` with falling `source_retention_acc` is the forgetting signature.